This notebook is tested on a serverless DBSQL cluster. Use an interactive cluster to create the widgets first and then switch to DBSQL.

# Fine Grained Demand Forecasting

*Prerequisite: Make sure to run 01_Introduction_And_Setup before running this notebook.*

In this notebook we use `AI_Forecast()` to forecast the SKU level demand

Key highlights for this notebook:
- `AI_Forecast()` is used to perform forecasting on time series data.   

In [0]:
%sql
--CREATE THIS ON A INTERACTIVE CLUSTER FIRST
--CREATE WIDGET TEXT catalogName DEFAULT "maxkoehler_demos";
--CREATE WIDGET TEXT dbName DEFAULT "demand_db";

In [0]:
%sql
USE CATALOG IDENTIFIER(:catalogName);
USE SCHEMA IDENTIFIER(:dbName);


## Using Databricks SQL AI Forecast Function
`AI_Forecast()` is a table-valued function designed to extrapolate time series data into the future

[AI_Forecast Reference](http://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)

The `AI_Forecast()` function uses a forecasting procedure described as a *prophet-like piecewise linear and seasonal model*. This model incorporates seasonality and assumes a linear trend over time.

At the moment of writing this it, is the only supported forecasting method currently available for this function.


###Check AI_Forecast() Function Capabilities
Using `AI_Forecast()` Function to forecast historical data for one SKU to show forecast quality for a specific SKU. 

In [0]:
%sql
select * from part_level_demand;

In [0]:
%sql
select distinct Product from part_level_demand;

In [0]:
%sql
WITH input_data AS (
  SELECT Date, Demand, SKU FROM part_level_demand
  WHERE SKU = (SELECT SKU FROM part_level_demand LIMIT 1) 
  AND Date <= (SELECT DATE_ADD(MIN(Date), 365*2) FROM part_level_demand)
)
SELECT forecast.Date, forecast.SKU, Demand, Demand_Forecast FROM AI_FORECAST(
    TABLE(input_data),
    horizon => DATE_ADD(DAY, 40, (SELECT MAX(Date) FROM part_level_demand)) ,
    time_col => 'Date',
    value_col => 'Demand',
    group_col => 'SKU',
    frequency => 'week',
    parameters => '{ "global_floor": 0, "yearly_order": 12 }'
) AS forecast
LEFT JOIN part_level_demand AS demand
ON forecast.Date = Demand.Date AND forecast.SKU = demand.SKU 
ORDER BY Date ASC

## Perform demand forecasts using AI_Forecast() function

Apply the `AI_Forecast()` function to our whole dataset to generate demand forecasts for the next 40 days and store the results into a delta table.

In [0]:
%sql
CREATE OR REPLACE TABLE part_level_demand_with_forecasts AS (
SELECT forecast.Date, forecast.SKU, demand.Product, forecast.Demand_Forecast AS Demand, TRUE AS is_forecast FROM AI_FORECAST(
    TABLE(SELECT Date, Demand, SKU FROM part_level_demand),
    horizon => DATE_ADD(DAY, 90, (SELECT MAX(Date) FROM part_level_demand)),
    time_col => 'Date',
    value_col => 'Demand',
    group_col => 'SKU',
    frequency => 'week',
    parameters => '{ "global_floor": 0, "yearly_order": 12 }'
) as forecast
LEFT JOIN (SELECT DISTINCT SKU, Product FROM part_level_demand) AS demand
ON forecast.SKU = demand.SKU
UNION ALL 
SELECT Date, SKU, Product, Demand, FALSE as is_forecast FROM part_level_demand
ORDER BY SKU, Date
)

In [0]:
%sql
SELECT * FROM part_level_demand_with_forecasts;